# Content

1. Introduction
2. TCR

    2.1. Unsupervised Clustering
    
    2.2. Database Queries
    2.3. Distance Measurements
    2.4. Epitope Prediction
3. BCR
    3.1. 

1. Introduction

B- and T-cells recognize their targets via their immune receptors (IRs) - B cell receptor (BCR), and T cell receptor (TCR). Their specificity is determined via the amino acid sequence. Previous studies provided evidence, that the most influencing factor of IR-target interaction the CDR3 sequence of the beta-chain and to lesser degree of the CDR3-alpha chain.

In many studies, determining the individual cell's specificity is of key interest: researcher's can then select the cells relevant to their study and observe their behaviour. Binding screening of T cells can be performed via ??? (Check whether similar for BCRs) (see ...). However, this adds another level of complexity onto study design and further increases cost. 
Computational approaches for inferring specificity can provide this layer of information.

Here, we will introduce the following approaches:
- Database Query: IR sequences and their targets from various studies were collected in multiple databases. We can use these to find matches to our single-cell study.
- Clustering and distances: [] showed, that IRs with similar receptors have common specificity. This property has been used in multiple approaches for comparing IRs with distance metrices and unsupervised clustering.
- Epitope prediction: Recently, several machine-learning methods were developed that directly predict binding between IRs and a target. In theory, these methods could be used to directly assign specificity to the IRs involved in single-cell studies.

However, all three approaches have major pitfalls. The amount of samples in the public databases is severly biased towards diseases and use cases that are commonly researched. Examplatory, this leads to known bindings for only several 100 epitopes sequences for TCRs in the major public databases. Further, the a majority amount of samples in these databases does not provide the full IR sequence (VDJ-genes and CDRs for alpha and beta chains), but rather focuses on the CDR3, while often only reporting only alpha or beta sequence. 

2. TCR
usage of tools depends on availability of underlying data, there not only the current study, but also the databases to use e.g. IEDB example. => ERGO-II reported for prediction: cdr3b > ...
It is to be assumed that this or similar order also hold not only for prediction but also for clustering, querying, and distance metrics.

2.1. Unsupervised Clustering
While not directly infering specificity => clusters of similar TCR sequences, likely to have similar specificities => few known TCR-epitope paires in data => assign specificity to rest of cluster


2.2. Database Queries
Idea => query databases for already reported TCR sequences:
common databases:
iedb
vdjdb
macpas
immuneaccess (covid)
=> other databases and studies with public data also possible

Tradeof between precision and recall as in clonotype definition: 
Query databases with various degress of overlap:
identical cdr3 beta, vdj genes, alpha chain
=> No rulebook yet to performance also highly case dependent (rare epitopes will require less strigent query for hits)

2.3. Distance metrics

If no direct hits, we can infer specificity by searching for similar TCRs within the databases.
Several distance metrics exist. 
In general three directions: 
sequence alignments => convert sequence between each other => most direct way to measure sequence similarity, gaps might interupt sequence motifs needed for TCR-Epitope interactions =>  very local search, 
kmer => similarity based on small motif segments of length k => capture preserved motifs within TCR sequences, Overall less similar sequences due to random reordering, maybe more global metric 
numeric embeddings => project TCR sequence into embedding space where nummeric distances can be calculated. => algorithm dependend, deep learning => infer this from data, however more black-box, less interpretable

Since no large benchmark between theese metrics exist, on tests showed generally similar performace, we will focus on one aligment based + 1 kmer based with two different requirements of used input data:
TCRdist for paired data => one of commenly used, requires alpha, beta + v genes 
TCRmatch for beta data alone => more recent method,  integrated into iedb

2.4. Prediction
This alternative approach not directly relying on databases. Prediction of binding between TCR sequence and epitope directly with measuring similarity to TCRs of databases. If in study there is guess of epitope (eg. due to prior vaccination) we can try to predict binding.
Latetly many machine learning models for predicting binding between TCR and epitope. Generally these methods perform well for epitopes with training data, but often fail to generalize for unseen epitopes.
Therefore recommend, to investigate training data of tool in question for the epitope in question, to see whether it was contained in training data to large extend. If not these tools can provide hint, but should still be taken with caution until they are further evaluated in a standarized benchmark for their individual strengths and weaknesses.

3. BCR
Similiar for TCR also on BCR

3.1. Clustering

3.2. Databases

In [ ]:
http://opig.stats.ox.ac.uk/webapps/covabdab/
https://www.iedb.org/

3.3. Distance Measurements

3.4. Prediction 

4. Takeaways